# Ground Truth getting started

- reference : https://aws.amazon.com/getting-started/hands-on/build-training-datasets-amazon-sagemaker-ground-truth/

## 1. 데이터셋 준비

In [1]:
import sagemaker
sess = sagemaker.session.Session()

# Prepare data and upload to your S3 bucket
BUCKET = sess.default_bucket()
EXP_NAME = 'ground-truth-demo' # Any valid S3 prefix.

다음 코드는 s3로부터 테스트용 이미지를 복사하여 SageMaker 디폴트 버킷에 저장합니다. (실행에 수 분 정도 소요됩니다.)

In [2]:
import itertools
import boto3

# Download and process the Open Images annotations
!wget https://storage.googleapis.com/openimages/2018_04/test/test-annotations-human-imagelabels-boxable.csv -O openimgs-annotations.csv
with open('openimgs-annotations.csv', 'r') as f:
    all_labels = [line.strip().split(',') for line in f.readlines()]
    
 # Extract image ids in each of our desired classes
ims = {}
ims['Truck'] = [label[0] for label in all_labels if (label[2] == '/m/07r04' and label[3] == '1')][:300]
ims['Limousine'] = [label[0] for label in all_labels if (label[2] == '/m/01lcw4' and label[3] == '1')][:300]
ims['Van'] = [label[0] for label in all_labels if (label[2] == '/m/0h2r6' and label[3] == '1')][:300]
ims['Car'] = [label[0] for label in all_labels if (label[2] == '/m/0pg52' and label[3] == '1')][:300]
ims['Motorcycle'] = [label[0] for label in all_labels if (label[2] == '/m/04_sv' and label[3] == '1')][:300]
num_classes = len(ims)

for key in ims.keys():
    ims[key] = set(ims[key])

# Create a new bucket for images to be labeled
s3 = boto3.client('s3')
sess = boto3.session.Session()
region = sess.region_name
if (region == 'us-east-1'):
    s3.create_bucket(Bucket=BUCKET)
else:
    s3.create_bucket(Bucket=BUCKET, 
                        CreateBucketConfiguration={'LocationConstraint': region})

# Copy the images to your local bucket
for img_id, img in enumerate(itertools.chain.from_iterable(ims.values())):
    copy_source = {
        'Bucket': 'open-images-dataset',
        'Key': 'test/{}.jpg'.format(img)
    }
    s3.copy(copy_source, BUCKET, '{}/images/{}.jpg'.format(EXP_NAME, img))

--2021-07-17 13:21:27--  https://storage.googleapis.com/openimages/2018_04/test/test-annotations-human-imagelabels-boxable.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.9.208, 172.217.12.240, 172.217.13.240, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.9.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26460276 (25M) [text/csv]
Saving to: ‘openimgs-annotations.csv’

openimgs-annotation 100%[===================>]  25.23M  89.9MB/s    in 0.3s    

2021-07-17 13:21:27 (89.9 MB/s) - ‘openimgs-annotations.csv’ saved [26460276/26460276]



S3에 저장된 이미지는 다음과 같습니다. (1014장의 이미지가 복사되었습니다.)

In [11]:
!aws s3 ls s3://{BUCKET}/{EXP_NAME}/images/ |head -5

2021-07-17 13:22:25     581486 000aa0b1c8fd5ddf.jpg
2021-07-17 13:21:55     132505 000b14e4ee4a2b2b.jpg
2021-07-17 13:23:38     166471 0013a0927e6bbefc.jpg
2021-07-17 13:22:30     195847 004b17294aff80dc.jpg
2021-07-17 13:21:31     305583 008168cfd6dcf2aa.jpg

[Errno 32] Broken pipe
Exception ignored in: <_io.TextIOWrapper name='<stdout>' mode='w' encoding='UTF-8'>
BrokenPipeError: [Errno 32] Broken pipe


In [12]:
!aws s3 ls s3://{BUCKET}/{EXP_NAME}/images/ | wc 

   1014    4056   52728


## 2. Private 팀 생성

### 콘솔에서 팀 생성 및 초대
Sagemaker 콘솔로 이동하여 private team을 만듭니다. `Groundt Truth`메뉴에서 `Private`탭을 클릭한 후 `Create private team`버튼을 클릭합니다.  

<img src="images/private-team.png" width=80% height=80%>

아래 그림을 참고하여 이메일로 팀원을 초대하고 팀을 구성합니다.  

<img src="images/team-config.png" width=70% height=70%>


### 이메일을 확인하고 패스워드를 설정합니다.

초대메일로 one-time 패스워드가 발송되었을 것입니다. https:// 로 시작하는 링크를 클릭한 후 전달받은 패스워드로 접속하고 새로운 패스워드를 입력합니다.

![](images/signin.png)



## 3. 레이블 작업 생성

아래 링크의 Step3부터 가이드를 참조하여 레이블작업을 설정하고 실행해봅니다. 

<span style="color:red"> NOTE : Step3의 g.단계에서 Mechanical Turk 가 아닌 private team을 선택합니다. 나머지 설정은 동일합니다. </span>

https://aws.amazon.com/getting-started/hands-on/build-training-datasets-amazon-sagemaker-ground-truth/

